In [128]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

import evaluator

from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [129]:
calendar_path = os.path.join(os.getcwd(), 'data', 'calendar.csv')
trainDF_path = os.path.join(os.getcwd(), 'data', 'sales_train_validation.csv')
pricesDF_path = os.path.join(os.getcwd(), 'data', 'sell_prices.csv')

df = pd.read_csv(trainDF_path)
calendarDF = pd.read_csv(calendar_path)
pricesDF = pd.read_csv(pricesDF_path)

In [355]:

# validation starts at 1885, so training # days is validation-dayStart
dayStart = 1155


dfTrainFull = df.iloc[:, :-28]
dfValid = df.iloc[:, -28:].copy()
dfTrain = df.iloc[:, 6+(dayStart-1):-28]
dfLabels = df.iloc[:, :6]

# pd.concat([dfLabels, dfTrain], axis=1)

In [356]:
# prepare X data 

# make dummies
xDF = pd.get_dummies(calendarDF.fillna(0), columns=['event_name_1', 'event_name_2', 'wday','month'])
# xDF = pd.get_dummies(calendarDF.fillna(0), columns=['event_name_1', 'event_name_2', 'wday'])

# merge duplicate events
xDF['event_name_1_Easter'] = xDF['event_name_1_Easter'] + xDF['event_name_2_Easter']
xDF['event_name_1_Cinco De Mayo'] = xDF['event_name_1_Cinco De Mayo'] + xDF['event_name_2_Cinco De Mayo']
xDF['event_name_1_OrthodoxEaster'] = xDF['event_name_1_OrthodoxEaster'] + xDF['event_name_2_OrthodoxEaster']
xDF["event_name_1_Father's day"] = xDF["event_name_1_Father's day"] + xDF["event_name_2_Father's day"]

# remove duplicates
columnList = ['event_name_1_Chanukah End', 'event_name_1_Christmas',
       'event_name_1_Cinco De Mayo', 'event_name_1_ColumbusDay',
       'event_name_1_Easter', 'event_name_1_Eid al-Fitr',
       'event_name_1_EidAlAdha', "event_name_1_Father's day",
       'event_name_1_Halloween', 'event_name_1_IndependenceDay',
       'event_name_1_LaborDay', 'event_name_1_LentStart',
       'event_name_1_LentWeek2', 'event_name_1_MartinLutherKingDay',
       'event_name_1_MemorialDay', "event_name_1_Mother's day",
       'event_name_1_NBAFinalsEnd', 'event_name_1_NBAFinalsStart',
       'event_name_1_NewYear', 'event_name_1_OrthodoxChristmas',
       'event_name_1_OrthodoxEaster', 'event_name_1_Pesach End',
       'event_name_1_PresidentsDay', 'event_name_1_Purim End',
       'event_name_1_Ramadan starts', 'event_name_1_StPatricksDay',
       'event_name_1_SuperBowl', 'event_name_1_Thanksgiving',
       'event_name_1_ValentinesDay', 'event_name_1_VeteransDay', 
       'wday_1', 'wday_2', 'wday_3', 'wday_4', 'wday_5', 'wday_6', 'wday_7',
       'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12']
xDF = xDF[columnList]

dayNum = pd.DataFrame(np.arange(1,len(xDF)+2),columns=['dayNum'])

xDF['dayNum'] = dayNum


In [361]:


ydfPred = pd.DataFrame(columns=dfValid.columns.tolist())

xTrain = xDF.iloc[dayStart-1:dayStart-1+dfTrain.shape[1],:]
xTest = xDF.iloc[dayStart-1+dfTrain.shape[1]:dayStart-1+dfTrain.shape[1]+28,:]

for i in range(0,int(len(dfTrain))):

    yTrain = dfTrain.iloc[i,:]

    regressor = LinearRegression()  
    regressor.fit(xTrain, yTrain)

    yPred = regressor.predict(xTest)

    ydfPred = ydfPred.append(pd.Series(yPred.round(), index=dfValid.columns.tolist()), ignore_index=True)

    if i % 1000 == 0:
        print(i)
out of to make sure it isn't hung# 

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000


In [362]:
# input dimensions must be 30490 x 28
e = evaluator.WRMSSEEvaluator(dfTrainFull, dfValid, calendarDF, pricesDF)

92%|█████████▏| 11/12 [00:00<00:00, 37.02it/s]

In [363]:
# score results - as of 3/24 current leaders are at something like .55
e.score(ydfPred)

# # totally naive "last 28 days"
# e.score(dfTrain.iloc[:,-28:].rename(columns=dict(zip(dfTrain.iloc[:,-28:].columns.tolist(), dfValid.columns.tolist()))))

0.871014916611208

In [364]:
ydfPred

,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,2.0,3.0,4.0,1.0,1.0,1.0,...,2.0,3.0,4.0,1.0,1.0,1.0,1.0,2.0,3.0,4.0
4,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,...,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
30486,-0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30487,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,...,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
30488,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,...,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0
